In [3]:
# %matplotlib widget
import os
import json
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.rc('font', family='Times New Roman')
from matplotlib import pyplot as plt
import scipy as sp
from scipy import signal as sg
from scipy import fft
from scipy import stats as sts
from scipy.io import wavfile
from fitter import Fitter, get_distributions
from collections import deque
from tqdm.notebook import tqdm, trange
from functools import partial
# import cv2 as cv

In [2]:
def spectrum(data: np.ndarray, sr: int):
    sp = np.abs(fft.fft(data))**2
    sp = fft.fftshift(sp)
    freq = fft.fftfreq(sp.size, 1 / sr)
    freq = fft.fftshift(freq)
    N = len(data)
    # figure = plt.figure(figsize=(16, 9))
    # plt.plot(freq[N // 2 : ], sp[N // 2 : ])
    # plt.title('Spectrum')
    return freq[N // 2:], sp[N // 2:]

    # welfreq, welsp = sg.welch(data, sr, 'hann', sr / 60, sr / 180, return_onesided = True, scaling='spectrum')
    # figure = plt.figure(figsize=(16, 9))
    # plt.plot(welfreq, 20 * np.log10(welsp))
    # return welfreq, 20 * np.log10(welsp)

In [ ]:
def bat_inter_new(filename: str, win_len_sec: float, step_len_sec: float, percentage_ACI :float, percentage_BPC:float,
                   stop_min: int = None) -> np.ndarray:
    sr, data = wavfile.read(filename)
    if stop_min is not None:
        data = data[:stop_min * 60 * sr]
    win_len = int(win_len_sec * sr)
    step_len = int(step_len_sec * sr)

    ord = 4
    low_freq = 45e3 / (sr / 2)
    high_freq = 60e3 / (sr / 2)
    num, denom = sg.butter(ord, [low_freq, high_freq], 'bandpass')
    flag = False
    stack = deque()
    temp = np.array([])
    seg_mean = np.array([])

    ACI_dist_params = (SOME_VALUE, SOME_VALUE)
    ACI_thresh = sts.levy.interval(percentage_ACI, *ACI_dist_params)[1]
    BPC_dist_params = (SOME_VALUE, SOME_VALUE)
    BPC_thresh = sts.cauchy.interval(percentage_BPC, *BPC_dist_params)[1]
    
    for i in trange(int(data.size // step_len)):
        clip = data[int(step_len * i):int(step_len * i + win_len)]
        n_clip = clip / np.iinfo(clip.dtype).max
        fil_curr_clip = sg.filtfilt(num, denom, n_clip)
        f, t, Zxx = sg.stft(n_clip, sr, 'hann', sr / 100, sr / 200, return_onesided=True, scaling='spectrum')
        spg_df = pd.DataFrame(np.abs(Zxx)**2, f, t)

        # mving_avg_win_len = 5
        ACI = (spg_df.diff(-1, 1).abs().sum(1) / spg_df.sum(1))
        # ACI -= ACI.mean()
        # ACI_mving_avg = ACI.rolling(mving_avg_win_len).mean()
        seg = ACI.iloc[np.where(f > 45e3)[0][0]:np.where(f < 60e3)[0][-1]]
        # print(seg.mean())

        # curr_clip_zc = np.where(np.diff(np.sign(ACI_mving_avg)))[0]
        # ACI_zc = np.where(np.diff(np.sign(seg)))[0]

        if seg.mean() > ACI_thresh and (np.sum(fil_curr_clip**2) / np.sum(n_clip**2)) > BPC_thresh:
            # seg_mean = np.append(seg_mean, seg.mean())
            if stack[-1] == 0:
                stack.append(-1)

            if flag:
                stack.pop()
                stack.pop()
                stack.append((step_len * i + win_len) / sr)
                stack.append(-2)
                flag = True
            else:
                temp = np.array(stack)
                temp = temp[(temp != -2) & (temp != -1) & (temp != 0)]
                if (temp.size > 1) and (temp[-1] > (step_len * i / sr)):
                    continue
                stack.append(step_len * i / sr)
                stack.append((step_len * i + win_len) / sr)
                stack.append(-2)
                flag = True
        else:
            stack.append(0)
            flag = False

    thresh_stack = np.array(stack)
    thresh_stack = thresh_stack[(thresh_stack != -2) & (thresh_stack != -1) &
                                (thresh_stack != 0)]
    thresh_stack_val_cnts = pd.value_counts(thresh_stack).sort_index()
    thresh_stack = thresh_stack_val_cnts.mask(thresh_stack_val_cnts != 1).dropna().index.to_numpy()
    redun_idx = np.where(
        (np.diff(thresh_stack[1:-1].reshape(-1, 2)) < win_len_sec +  step_len_sec) == True)[0] + 1
    redun_idx = np.array([redun_idx * 2 - 1, redun_idx * 2]).flatten()
    return np.delete(thresh_stack, redun_idx).reshape(-1, 2)# , seg_mean

In [ ]:
interval_logs = bat_inter_new(f'Audio.wav', 0.5, 0.1, 0.55, 0.25, 5)
# pd.DataFrame(interval_logs, columns=['Start Time', 'End Time']).to_csv(f'file.csv')

<h1>ACI</h1>

In [ ]:
sr, data = wavfile.read('Audio.wav')

ord = 4
low_freq = 45e3 / (sr / 2)
high_freq = 60e3 / (sr / 2)
num, denom = sg.butter(ord, [low_freq, high_freq], 'bandpass')
win_len = int(0.5 * sr)
step_len = int(0.1 * sr)

info = np.array([])

for i in trange(int(data.size // win_len)):
    clip = data[int(win_len * i):int(win_len * (i + 1))]
    n_clip = clip / np.iinfo(clip.dtype).max
    f, t, Zxx = sg.stft(n_clip, sr, 'hann', sr / 100, sr / 200, return_onesided=True)
    spg_df = pd.DataFrame(np.abs(Zxx), f, t)
    ACI = (spg_df.diff(-1, 1).abs().sum(1) / spg_df.sum(1))
    seg = ACI.iloc[np.where(f > 45e3)[0][0]:np.where(f < 60e3)[0][-1]]
   
    info = np.append(info, [ACI.mean(), seg.mean()])


In [ ]:
# dist_name = ['levy', 'fatiguelife', 'reciprocal', 'loguniform', 'johnsonsb']
# dist_name = get_distributions()
f_50 = Fitter(infoo[:, 1], distributions=dist_name, bins=50, timeout=120)
f_50.fit()
f_50.summary()

<h1>BPC</h1>

In [6]:
filename = 'Audio.wav'
sr, data = wavfile.read(filename)
data = data / np.iinfo(data.dtype).max
ord = 4
low_freq = 45e3 / (sr / 2)
high_freq = 60e3 / (sr / 2)
num, denom = sg.butter(ord, [low_freq, high_freq], 'bandpass')
data = pd.Series(data[: int(data.shape[0] // sr) * sr])
filted_data = pd.Series(sg.filtfilt(num, denom, data))**2
data = data ** 2
pow_df = (data).rolling(window = int(sr * 0.5), step = int(sr * 0.5)).sum().dropna().reset_index(drop=True)
filted_pow_df = (filted_data).rolling(window = int(sr * 0.5), step = int(sr * 0.5)).sum().dropna().reset_index(drop=True)

In [ ]:
# dist_name = ['cauchy', 'alpha', 'invgauss', 'norminvgauss', 'foldcauchy']
# dist_name = get_distributions()
f = Fitter(filted_pow_df / pow_df, distributions= dist_name, bins=50, timeout=120)
f.fit()
f.summary()